## Классификация стихотворений с использованием NLP
Botasheva Zhanna

In [4]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import *
import warnings
warnings.filterwarnings("ignore")

In [5]:
import nltk   # Natural Language Toolkit
from nltk.corpus import stopwords
from string import punctuation
from pymystem3 import Mystem

In [6]:
from google.colab import drive

# Монтируем Google Диск
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#df_train = pd.read_csv('C:\\Users\\Zhanna Botasheva\\Desktop\\Курс ML\\NLP\\train_data.csv')
#df_test = pd.read_csv('C:\\Users\\Zhanna Botasheva\\Desktop\\Курс ML\\NLP\\test_data.csv')
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/train_data.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/test_data.csv')

In [8]:
df_train.head()

,Genre,Poem
0,Music,NaN
1,Music,In the thick brushthey spend the...
2,Music,Storms are generous. ...
3,Music,—After Ana Mendieta Did you carry around the ...
4,Music,for Aja Sherrard at 20The portent may itself ...


In [9]:
df_test.head()

,Genre,Poem
0,Music,A woman walks by the bench I’m sitting onwith ...
1,Music,"Because I am a boy, the untouchability of beau..."
2,Music,"Because today we did not leave this world,We n..."
3,Music,"Big Bend has been here, been here. Shouldn’t i..."
4,Music,"I put shells there, along the lip of the road...."


In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 841 entries, 0 to 840
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Genre   841 non-null    object
 1   Poem    837 non-null    object
dtypes: object(2)
memory usage: 13.3+ KB


In [11]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Genre   150 non-null    object
 1   Poem    150 non-null    object
dtypes: object(2)
memory usage: 2.5+ KB


Избавимся от Nan в Poem

In [12]:
#df['Рейтинг'] = df['Рейтинг'].fillna(0)
df_train = df_train.dropna(subset=['Poem'])

In [13]:
df_train.head()

,Genre,Poem
1,Music,In the thick brushthey spend the...
2,Music,Storms are generous. ...
3,Music,—After Ana Mendieta Did you carry around the ...
4,Music,for Aja Sherrard at 20The portent may itself ...
5,Music,"for Bob Marley, Bavaria, November 1980 Here i..."


In [14]:
df_train['Genre'].value_counts(dropna=False)

,count
Genre,
Music,238
Death,231
Environment,227
Affection,141


In [15]:
df_test['Genre'].value_counts(dropna=False)

,count
Genre,
Affection,100
Environment,25
Death,13
Music,12


In [16]:
df=pd.concat([df_train,df_test],axis=0)
df

,Genre,Poem
1,Music,In the thick brushthey spend the...
2,Music,Storms are generous. ...
3,Music,—After Ana Mendieta Did you carry around the ...
4,Music,for Aja Sherrard at 20The portent may itself ...
5,Music,"for Bob Marley, Bavaria, November 1980 Here i..."
...,...,...
145,Environment,"To pick a tulip from the garden, the red one. ..."
146,Environment,We are as clouds that veil the midnight moon; ...
147,Environment,"When pulled, the spider web took another form...."
148,Environment,Whose woods these are I think I know. His hous...


In [17]:
df['Genre'].value_counts(dropna=False)

,count
Genre,
Environment,252
Music,250
Death,244
Affection,241


Целевая переменная сбалансирована

Удалим все нерелевантные символы

In [18]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "", regex=True)
    df[text_field] = df[text_field].str.replace(r"http", "", regex=True)
    df[text_field] = df[text_field].str.replace(r"@\S+", "", regex=True)
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ", regex=True)
    df[text_field] = df[text_field].str.replace(r"#", "", regex=True)
    df[text_field] = df[text_field].str.replace(r"@", "at", regex=True)
    df[text_field] = df[text_field].str.replace(r"%20", "at", regex=True)
    df[text_field] = df[text_field].str.lower()
    return df

df_test = standardize_text(df, "Poem")

df.to_csv("clean_train.csv")
df.head(5)

,Genre,Poem
1,Music,in the thick brushthey spend the...
2,Music,storms are generous ...
3,Music,after ana mendieta did you carry around the ...
4,Music,for aja sherrard at 20the portent may itself ...
5,Music,"for bob marley, bavaria, november 1980 here i..."


Разобьем выборку на две части: тренировочную (train) и тестовую выборку (test). На test будем проверять обобщающая способность модели

In [19]:
from sklearn.model_selection import train_test_split as tts

X_train,X_test = tts(df,test_size=0.3,random_state=42)

In [20]:
# Тренировачная выборка

corpus_tr = list(X_train['Poem'])  # корпус
target_tr = list(X_train['Genre'])   # целевая функция

In [21]:
# Тестовая выборка

corpus_te = list(X_test['Poem'])  # корпус
target_te = list(X_test['Genre'])   # целевая функция

### Варианты TF-IDF
Варианты преобразования документов в корпусе используя TF-IDF для создания векторного представления слов перечислены ниже

In [22]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

english_stopwords = stopwords.words("english")

# минимальная частота слов
v1_vectoriser = TfidfVectorizer(min_df=3,norm=None)
v1_vectoriser.fit(corpus_tr)
X1 = v1_vectoriser.transform(corpus_tr)

v2_vectoriser = TfidfVectorizer(min_df=2,norm=None)
v2_vectoriser.fit(corpus_tr)
X2 = v2_vectoriser.transform(corpus_tr)

v3_vectoriser = TfidfVectorizer(min_df=1,norm=None)
v3_vectoriser.fit(corpus_tr)
X3 = v3_vectoriser.transform(corpus_tr)

In [24]:
# стоп слова варианты
v4_vectoriser = TfidfVectorizer(min_df=3,norm=None,stop_words=english_stopwords)
v4_vectoriser.fit(corpus_tr)
X4 = v4_vectoriser.transform(corpus_tr)

v5_vectoriser = TfidfVectorizer(min_df=2,norm=None,stop_words=english_stopwords)
v5_vectoriser.fit(corpus_tr)
X5 = v5_vectoriser.transform(corpus_tr)

v6_vectoriser = TfidfVectorizer(min_df=1,norm=None,stop_words=english_stopwords)
v6_vectoriser.fit(corpus_tr)
X6 = v6_vectoriser.transform(corpus_tr)

In [25]:
# н-грам варианты
v7_vectoriser = TfidfVectorizer(min_df=2,norm=None,stop_words=english_stopwords,ngram_range=(1,1))
v7_vectoriser.fit(corpus_tr)
X7 = v7_vectoriser.transform(corpus_tr)

v8_vectoriser = TfidfVectorizer(min_df=2,norm=None,stop_words=english_stopwords,ngram_range=(1,2))
v8_vectoriser.fit(corpus_tr)
X8 = v8_vectoriser.transform(corpus_tr)

v9_vectoriser = TfidfVectorizer(min_df=2,norm=None,stop_words=english_stopwords,ngram_range=(1,3))
v9_vectoriser.fit(corpus_tr)
X9 = v9_vectoriser.transform(corpus_tr)

v10_vectoriser = TfidfVectorizer(min_df=2,norm=None,stop_words=english_stopwords,ngram_range=(2,2))
v10_vectoriser.fit(corpus_tr)
X10 = v10_vectoriser.transform(corpus_tr)

v11_vectoriser = TfidfVectorizer(min_df=2,norm=None,stop_words=english_stopwords,ngram_range=(3,3))
v11_vectoriser.fit(corpus_tr)
X11 = v11_vectoriser.transform(corpus_tr)

v12_vectoriser = TfidfVectorizer(min_df=2,norm=None,stop_words=english_stopwords,ngram_range=(4,4))
v12_vectoriser.fit(corpus_tr)
X12 = v12_vectoriser.transform(corpus_tr)

In [26]:
# максимальная частота слов
v13_vectoriser = TfidfVectorizer(min_df=2,max_df=0.9,norm=None,stop_words=english_stopwords)
v13_vectoriser.fit(corpus_tr)
X13 = v13_vectoriser.transform(corpus_tr)

v14_vectoriser = TfidfVectorizer(min_df=2,max_df=0.8,norm=None,stop_words=english_stopwords)
v14_vectoriser.fit(corpus_tr)
X14 = v14_vectoriser.transform(corpus_tr)

v15_vectoriser = TfidfVectorizer(min_df=2,max_df=0.7,norm=None,stop_words=english_stopwords)
v15_vectoriser.fit(corpus_tr)
X15 = v15_vectoriser.transform(corpus_tr)

### Варианты Моделей

Начнем с базового RandomForest (не очень глубокого) <code>model_srf</code>, а потом попробуем улучшить результат с помощью <code>model_drf</code> и <code>model_ocb</code>

In [27]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.7 MB/s eta 0:00:00


In [28]:
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

model_srf = RandomForestClassifier(n_estimators=10,random_state=32)  # не глубокий случайный лес
model_drf = RandomForestClassifier(n_estimators=40,random_state=32)  # глубокий случайный лес
model_ocb = CatBoostClassifier(silent=True)  # градиентный бустинг

In [29]:
from sklearn.metrics import accuracy_score

# вспомогательная функция для оценки модели

def evaluate_tfidf(X,vect,name,model):

    print(f'case_id: :{name}')
    print('==================================')

    # train model
    model.fit(X,target_tr)
    y_model = model.predict(X)
    print(f'train: {accuracy_score(target_tr,y_model)}')

    X = vect.transform(corpus_te)
    y_model = model.predict(X)
    print(f'test: {accuracy_score(y_model,target_te)}')

    print('==================================','\n')

### Минимальная частота слов

Проверим как влияет фильтр минимальной частоты слов (<code>min_df</code>) в документе на результат
- <code>v1</code> (<code>min_df=3</code> фильтруем слова которые появляются в документов меньше 3 раз)
- <code>v2</code> (<code>min_df=2</code> фильтруем слова которые появляются в документов меньше 2 раз)
- <code>v3</code> (<code>min_df=1</code> фильтруем слова которые появляются в документов меньше 1 раза)

Как мы видим, эффект от фильтрации часто встречающихся слов не сильно разный

In [30]:
evaluate_tfidf(X3,v3_vectoriser,'v3',model_srf)
evaluate_tfidf(X2,v2_vectoriser,'v2',model_srf)
evaluate_tfidf(X1,v1_vectoriser,'v1',model_srf)

case_id: :v3
train: 0.9391304347826087
test: 0.31986531986531985

case_id: :v2
train: 0.9449275362318841
test: 0.32323232323232326

case_id: :v1
train: 0.9391304347826087
test: 0.3265993265993266



### Стоп Слова

- Попробуем повысить точность с помощью фильтрации не релевантных слов (стоп слова)
- При использовании разных настроек <code>min_df</code> точность модели меняется по разному; <code>min_df=3</code> дает наиболее хороший результат

Вариатны:
- <code>v4</code> (фильтрация стоп слов с <code>min_df=3</code>)
- <code>v5</code> (фильтрация стоп слов с <code>min_df=2</code>)
- <code>v6</code> (фильтрация стоп слов с <code>min_df=1</code>)

In [31]:
evaluate_tfidf(X4,v4_vectoriser,'v4',model_srf)
evaluate_tfidf(X5,v5_vectoriser,'v5',model_srf)
evaluate_tfidf(X6,v6_vectoriser,'v6',model_srf)

case_id: :v4
train: 0.9376811594202898
test: 0.4006734006734007

case_id: :v5
train: 0.936231884057971
test: 0.38047138047138046

case_id: :v6
train: 0.9391304347826087
test: 0.367003367003367



Вариант <code>v4</code> дает нам точность 0.4 на тестовой выборке, используем этот вариант и проверим разные комбинации н-грамм

###  н-граммы



Варианты:

- <code>v7</code> ngram_range=(1,1) (Только униграммы)
- <code>v8</code> ngram_range=(1,2) (Униграммы и биграммы)
- <code>v9</code> ngram_range=(1,3) (униграммы, биграммы и триграммы)

In [32]:
evaluate_tfidf(X9,v9_vectoriser,'v9',model_srf)
evaluate_tfidf(X8,v8_vectoriser,'v8',model_srf)
evaluate_tfidf(X7,v7_vectoriser,'v7',model_srf)

case_id: :v9
train: 0.936231884057971
test: 0.39057239057239057

case_id: :v8
train: 0.9449275362318841
test: 0.3838383838383838

case_id: :v7
train: 0.936231884057971
test: 0.38047138047138046



Проверим еще варианты:

- <code>v10</code> ngram_range=(1,1) (Только униграммы)
- <code>v11</code> ngram_range=(2,2) (Только биграммы)
- <code>v12</code> ngram_range=(3,3) (Только триграммы)

In [33]:
evaluate_tfidf(X12,v12_vectoriser,'v12',model_srf)
evaluate_tfidf(X11,v11_vectoriser,'v11',model_srf)
evaluate_tfidf(X10,v10_vectoriser,'v10',model_srf)

case_id: :v12
train: 0.2942028985507246
test: 0.2356902356902357

case_id: :v11
train: 0.30289855072463767
test: 0.24242424242424243

case_id: :v10
train: 0.5072463768115942
test: 0.21885521885521886



Как мы видим, улучшении никаких нет на тестовой выборке при использовании bigram,trigram, точность сильно снизилось,  базовые токены (unigram) являются критичным для высокой точности модели

### Максимальная частота

Максимальная частота слов <code>max_df</code> в векторизаторе, попробуем варианты:
- <code>v13</code> <code>max_df=0.9</code> (фильтруем слова которые появляются в 90% документов)
- <code>v14</code> <code>max_df=0.8</code> (фильтруем слова которые появляются в 80% документов)
- <code>v15</code> <code>max_df=0.7</code> (фильтруем слова которые появляются в 70% документов)

Как мы видим, улучшении никаких нет, при уменьшении часто встречаюшихся слов, точность падает

In [34]:
evaluate_tfidf(X13,v13_vectoriser,'v13',model_srf)
evaluate_tfidf(X14,v14_vectoriser,'v14',model_srf)
evaluate_tfidf(X15,v15_vectoriser,'v15',model_srf)

case_id: :v13
train: 0.936231884057971
test: 0.38047138047138046

case_id: :v14
train: 0.936231884057971
test: 0.38047138047138046

case_id: :v15
train: 0.936231884057971
test: 0.38047138047138046



### Выбор модели

- Базовая модель случайного леса не очень глубокая, соответственно может не очень хорошо определить закономерности в данных
- Воспользуемся вариантом, который показал наилучшую обобщающию способность на тестовой выборке (<code>v4</code>)
- CatBoost хорошо оптимизируется под любые табличные данные, и как мы видим улучшения нет, обе модели показывают примерно идентичный результат
- Соответственно возможно что улучшении на тестовой выборке с TF-IDF (помимо токенизации) особо нет, и нужно проверить эмбеддинговый подход

In [35]:
# 1314 признаков

X4.shape

(690, 1314)

In [36]:
evaluate_tfidf(X4,v4_vectoriser,'v4_deeprf',model_drf)
evaluate_tfidf(X4,v4_vectoriser,'v4_catboost',model_ocb)

case_id: :v4_deeprf
train: 0.9507246376811594
test: 0.4444444444444444

case_id: :v4_catboost
train: 0.927536231884058
test: 0.43434343434343436



### TF-IDF GridSearchCV

- В ручную подборка гиперпараметров может оказаться не самым оптимальным вариантов, поэтому воспользуемся <code>GridSearchCV</code> методом кросс-валидации для подборки гиперпараметров модели
- Попробуем найти более оптимальные параметры для модели случайного леса, так как CatBoost показал хуже результат

У случайного леса много параметров, которые можно перепробовать, ограничимся:
- <code>max_depth</code> максимальная глубина разбиения
- <code>n_estimators</code> количество решаущих деревьев
- <code>criterion</code> критерии оценки разбиения в узле
- <code>min_samples_leaf</code> минимальное количество образцов, необходимое для нахождения в листовом узле каждого дерева
- <code>min_samples_split</code> Минимальное количество образцов, необходимое для разбиения внутреннего узла каждого дерева

Из соображении времени, количество построенных деревьев ограничивается (e_stimators=100)

In [37]:
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint
import numpy as np


def evaluate_cv(X,name,base_estimator):

    print(f'case_id: :{name}')
    print('==================================')

    rs_space={'max_depth':list(np.arange(10,60,10)) + [None],
              'n_estimators':np.arange(40,120, step=20),
              'criterion':['gini','entropy'],
              'min_samples_leaf':[1,2,3,4],
               'min_samples_split':np.arange(2,6, step=2),
              }

    model = GridSearchCV(base_estimator,
                         rs_space,
                         scoring='accuracy',
                         n_jobs=-1,
                         cv=3)

    # train model
    model.fit(X,target_tr)

    print('Best hyperparameters are: '+str(model.best_params_))
    print('Best score is: '+str(model.best_score_))

    print('==================================','\n')



Кросс валидация нам дает немного лучше результат чем на train/test разбиение; в итоге мы получаем **точность 0.47** на тестовой выборке в cv

In [38]:
%%time

model_drf = RandomForestClassifier(random_state=32)
evaluate_cv(X4,'random_forest_cv',model_srf)

case_id: :random_forest_cv
Best hyperparameters are: {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 100}
Best score is: 0.4710144927536232

CPU times: user 3.69 s, sys: 416 ms, total: 4.11 s
Wall time: 3min 25s


Проверим точность на всей тестовой и отложенной тестовой выборке на больше объеме данных у нас модель показывает хуже результат

In [39]:
best_model = RandomForestClassifier(**{'criterion': 'entropy',
                                     'max_depth': 40,
                                     'min_samples_leaf': 3,
                                     'min_samples_split': 2,
                                     'n_estimators': 100,
                                     'random_state':32})

evaluate_tfidf(X4,v4_vectoriser,'v4_rf_best',best_model)

case_id: :v4_rf_best
train: 0.7594202898550725
test: 0.46464646464646464



### Эмбеддинги

Для нашей задачи возьмем предобученные эмбеддинги

In [40]:
# Не забываем про веса tfidf; создадим словарь
tfidf_weights = dict(zip(list(v4_vectoriser.vocabulary_.keys()),v4_vectoriser.idf_))

Распакуем эмбеддинги и загружаем вектора

In [41]:
import gensim.downloader as api

word2vec = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


### Токенизация документов

- Токенизируем все документы в тренировочной выборке/копрусе, будем использовать стандартный метод из nltk <code>word_tokenize</code>
- Токенизированные документы сохраним в <code>lst_corpus_tr</code> и <code>lst_corpus_te</code> для <code>train</code> и <code>test</code> выборки

In [42]:
from nltk.tokenize import word_tokenize

# Preprocessing, returns list instead
def tokenise_for_word2vec(text):

    text = text.lower() #changes to lower case
    tokens = word_tokenize(text) #tokenize the text

    clean_list = []
    for token in tokens:
        clean_list.append(token)

    return clean_list

In [43]:
import nltk

# Загрузка необходимых ресурсов
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [44]:
lst_corpus_tr = []
for doc in corpus_tr:
    lst_corpus_tr.append(tokenise_for_word2vec(doc))

lst_corpus_te = []
for doc in corpus_te:
    lst_corpus_te.append(tokenise_for_word2vec(doc))

### Усредненные Вектора (без TF-IDF весов)

- Воспользуемся самым простым подходом; для каждого документа мы возьмем **усредненный эмбеддинговый вектор** всех слов которые в него входят
- Размерность данных с эмбеддингами получается намного ниже (300) чем в TF-IDF (1775) что должно сказаться на скорости обучения модели
- Сохраняем эмбеддинг для каждого документа в <code>X_tr</code> и <code>X_te</code> для <code>train</code> и <code>test</code> выборки

In [45]:
# Get average embedding vector for each text

def doc_vectoriser(doc, model):

    doc_vector = []
    num_words = 0

    for word in doc:
        try:
            if num_words == 0:
                doc_vector = model[word]
            else:
                doc_vector = np.add(doc_vector, model[word])
            num_words += 1
        except:
            pass  # if embedding vector isn't found

    return np.asarray(doc_vector) / num_words


def doc_vectoriser(doc, model):
    doc_vector = np.zeros((300,))  # Предполагаем, что размерность эмбеддингов 300
    num_words = 0

    for word in doc:
        if word in model:
            doc_vector = np.add(doc_vector, model[word])
            num_words += 1

    # Избегаем деления на ноль
    return doc_vector / num_words if num_words > 0 else doc_vector

X_tr = np.array([doc_vectoriser(doc, word2vec) for doc in lst_corpus_tr])
X_te = np.array([doc_vectoriser(doc, word2vec) for doc in lst_corpus_te])

In [46]:
X_tr = []
for doc in lst_corpus_tr:
    X_tr.append(doc_vectoriser(doc,word2vec))


X_te = []
for doc in lst_corpus_te:
    X_te.append(doc_vectoriser(doc,word2vec))

### Усредненные Вектора (c TF-IDF весами)

- Мы сохранили веса значимости слов (токенов) в документах корпуса в <code>tfidf_weights</code>
- Сохраняем взвешенные эмбеддинг для каждого документа в <code>Xw_tr</code> и <code>Xw_te</code> для <code>train</code> и <code>test</code> выборки

In [47]:
# Get average embedding vector for each text

def doc_vectoriser_tfidf(doc, model):
    doc_vector = np.zeros((300,))  # Предполагаем, что размерность эмбеддингов 300
    num_words = 0

    for word in doc:
        if word in model:
            if word in tfidf_weights:
                doc_vector = np.add(doc_vector, model[word] * tfidf_weights[word])
            else:
                doc_vector = np.add(doc_vector, model[word])
            num_words += 1

    return doc_vector / num_words if num_words > 0 else doc_vector

Xw_tr = np.array([doc_vectoriser_tfidf(doc, word2vec) for doc in lst_corpus_tr])
Xw_te = np.array([doc_vectoriser_tfidf(doc, word2vec) for doc in lst_corpus_te])

In [48]:
Xw_tr = []
for doc in lst_corpus_tr:
    Xw_tr.append(doc_vectoriser_tfidf(doc,word2vec))

Xw_te = []
for doc in lst_corpus_te:
    Xw_te.append(doc_vectoriser_tfidf(doc,word2vec))

###  Варианты Моделей

Повторим то что мы делали в части I:

- В плане моделей, попробуем два подхода ансаблья, случайный лес (который использует бэггинт и метод случайного пространства) и градиентный бустинг СatBoost
- Начнем с базового RandomForest (не очень глубокого) <code>model_srf</code>, а потом попробуем улучшить результат с помошью <code>model_drf</code> и <code>model_ocb</code>

In [49]:
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

model_srf = RandomForestClassifier(n_estimators=10,random_state=32)
model_drf = RandomForestClassifier(n_estimators=40,random_state=32)
model_ocb = CatBoostClassifier(silent=True)

In [50]:
from sklearn.metrics import accuracy_score

# Импортируем матричу фич (эмбеддинги) X_tr, X_te
# target_tr, target_te глоб

def evaluate_embedding(X_tr, X_te, name, model):
    print(f'case_id: :{name}')
    print('==================================')

    try:
        # Train model
        model.fit(X_tr, target_tr)
        y_model_train = model.predict(X_tr)
        print(f'train: {accuracy_score(target_tr, y_model_train)}')

        y_model_test = model.predict(X_te)
        print(f'test: {accuracy_score(y_model_test, target_te)}')
    except Exception as e:
        print(f'Error during training or evaluation: {e}')

    print('==================================\n')


### Варианты Моделей (c TF-IDF весами)

- Модели **с TF-IDF весами** для эмбеддингов
- Точность с весами не улучшилась

In [51]:
evaluate_embedding(X_tr,X_te,'google-news',model_drf)

case_id: :google-news
train: 0.9536231884057971
test: 0.32323232323232326



In [52]:
evaluate_embedding(X_tr,X_te,'google-news',model_ocb)

case_id: :google-news
train: 0.9536231884057971
test: 0.4074074074074074



In [53]:
evaluate_embedding(Xw_tr,Xw_te,'google-news_rf_tfidf_weights',model_drf)

case_id: :google-news_rf_tfidf_weights
train: 0.9536231884057971
test: 0.3501683501683502



In [54]:
evaluate_embedding(Xw_tr,Xw_te,'google-news_cat_tfidf_weights',model_ocb)

case_id: :google-news_cat_tfidf_weights
train: 0.9536231884057971
test: 0.41414141414141414



### GridSearchCV (без TF-IDF весов)

- Воспользуемся той же функции что и раньше для того чтобы оптимизировать гиперпараметры модели
- Будем использовать случайны лес, и найдем наилучшие гиперпараметры используя кросс-валидацию <code>GridSearchCV</code>

In [55]:
%%time

model_drf = RandomForestClassifier(random_state=32)
evaluate_cv(np.array(X_tr),'google-news_rf_cv',model_drf)

case_id: :google-news_rf_cv
Best hyperparameters are: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 80}
Best score is: 0.44782608695652176

CPU times: user 4.85 s, sys: 658 ms, total: 5.51 s
Wall time: 8min 31s


In [56]:
best_model_rf = RandomForestClassifier(**{'criterion': 'entropy',
                                       'max_depth': 10,
                                       'min_samples_leaf': 1,
                                       'min_samples_split': 4,
                                       'n_estimators': 100,
                                       'random_state':32})

In [57]:
evaluate_embedding(X_tr,X_te,'google-news_rf_tfidf_weights',best_model_rf)

case_id: :google-news_rf_tfidf_weights
train: 0.9536231884057971
test: 0.40404040404040403



### GridSearchCV (c TF-IDF весами)

- Повторяем оптимизацию гиперпараметров для эмбеддингов с весами TF-IDF
- Умножив веса из TF-IDF на эмбеддинги модель не много улучшилась

In [58]:
%%time

model_drf = RandomForestClassifier(random_state=32)
evaluate_cv(np.array(Xw_tr),'google-news_rf_cv_tfidf_weights',model_drf)

case_id: :google-news_rf_cv_tfidf_weights
Best hyperparameters are: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 80}
Best score is: 0.4666666666666666

CPU times: user 5.43 s, sys: 660 ms, total: 6.09 s
Wall time: 8min 32s


Повторяем оптимизацию гиперпараметров для эмбеддингов с весами TF-IDF.

Умножив веса из TF-IDF на эмбеддинги не улучшило модель.